In [3]:
import os
import cv2
import shutil
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot  as plt
import sys
sys.path.append("../")

In [82]:
def load_yolo_labels(label_path, img_shape):
    """
    Load YOLO labels from a file and convert them to pixel coordinates.

    Inputs:
        - `label_path`: Path to the label file | type: str
        - `img_shape`: Shape of the image (height, width) | type: tuple
    Output(s):
        - List of labels in pixel coordinates | type: list
    """
    with open(label_path, "r") as f:
        lines = f.readlines()
    
    classes = []
    for i in range(len(lines)):
        # Convert string to numpy array and reshape
        classes.append(int(lines[i].split()[0]))
        lines[i] = np.array([float(j) for j in lines[i].replace("\n","").split()[1:]]).reshape((-1,2))
        # Convert normalized coordinates to pixel coordinates
        lines[i][:, 0] *= img_shape[0]
        lines[i][:, 1] *= img_shape[1]
        # Convert to integer
        lines[i] = lines[i].astype(np.int32)
    return classes, lines

def get_box_from_poly(poly, img_shape):
    xmin, ymin = poly.min(axis=0)
    xmax,  ymax = poly.max(axis=0)
    
    xc = np.clip((xmin+xmax)/2 / img_shape[1], 0, 1)
    yc = np.clip((ymin+ymax)/2 / img_shape[0], 0, 1)
    w = np.clip((xmax-xmin) / img_shape[1], 0, 1)
    h = np.clip((ymax-ymin) / img_shape[0], 0, 1)
    
    return xc, yc, w, h

In [85]:
roots = [Path(f"../TRAIN_DATA/cls_Letters/P1/"), Path(f"../TRAIN_DATA/cls_Letters/P2/")]
dests = [Path(f"../TRAIN_DATA/cls_Letters/Boxed/P1/"), Path(f"../TRAIN_DATA/cls_Letters/Boxed/P2/")]
for split in ["train", "val"]:
    for i, root in enumerate(roots):
        root = root/split
        dest = dests[i]/split
        os.makedirs(dest, exist_ok=True)
        (dest/"images").mkdir(exist_ok=True)
        (dest/"labels").mkdir(exist_ok=True)

        image_paths = glob(os.path.join(root, "images/*"))
        label_paths = [str(i).replace("images", "labels").replace(".jpeg", ".txt").replace(".jpg", ".txt") for i in image_paths]

        from custom_functions.yolo_data import draw_polygons
        for i,label_path in enumerate(label_paths):
            img = cv2.imread(image_paths[i])
            classes, polygons = load_yolo_labels(label_path, img.shape)
            label = ""


            # plt.imshow(draw_polygons(img, polygons))
            # plt.show()
            for j, poly in enumerate(polygons):
                box = get_box_from_poly(poly, img.shape)
                label += f"{classes[j]} {box[0]} {box[1]} {box[2]} {box[3]}\n"

            with open(dest/"labels"/Path(label_path).name, "w") as f:
                f.write(label)

            shutil.copy2(image_paths[i], dest/"images"/Path(image_paths[i]).name)